In [24]:
!pip install -q joblib pandas scikit-learn

import os
import io
from google.colab import files
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split

In [25]:
# ---------- Configuration (change if needed) ----------
SCALER_CHOICE = "standard"   # "standard", "minmax", or "robust"
DATE_COL = "Order Date"      # set to your date column name if present; otherwise set to None
REGION_COL = "Region"        # set to region/store column if present; otherwise set to None
TIME_SAFE = True             # True => fit scaler on training portion only (recommended for forecasting)
TEST_SIZE = 0.2              # fraction for test split when doing time-safe split per region

OUTPUT_DIR = "/content/scaling_output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [26]:
# ---------- Upload CSV (Colab file picker) ----------
print("Please upload your CSV file when prompted (choose stores_sales_forecasting.csv).")
uploaded = files.upload()
if not uploaded:
    raise SystemExit("No file uploaded. Re-run the cell and upload your CSV.")

# take first uploaded file
uploaded_filename = list(uploaded.keys())[0]
print("Uploaded file:", uploaded_filename)

Please upload your CSV file when prompted (choose stores_sales_forecasting.csv).


Saving stores_sales_forecasting.csv to stores_sales_forecasting.csv
Uploaded file: stores_sales_forecasting.csv


In [27]:
# ---------- Robust CSV loading (try typical encodings) ----------
def robust_read_csv_bytes(bytes_io):
    encodings = [None, "utf-8", "latin-1", "cp1252", "ISO-8859-1"]
    for enc in encodings:
        try:
            if enc is None:
                # try pandas auto-detect
                df_tmp = pd.read_csv(io.BytesIO(bytes_io))
            else:
                df_tmp = pd.read_csv(io.BytesIO(bytes_io), encoding=enc)
            print(f"Loaded using encoding: {enc or 'default'}")
            return df_tmp
        except Exception as e:
            # continue and try next encoding
            continue
    raise ValueError("Failed to read CSV with tried encodings.")

file_bytes = uploaded[uploaded_filename]
df = robust_read_csv_bytes(file_bytes)
print("Loaded dataframe shape:", df.shape)
print("Columns:", df.columns.tolist())

Loaded using encoding: latin-1
Loaded dataframe shape: (2121, 21)
Columns: ['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode', 'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State', 'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category', 'Product Name', 'Sales', 'Quantity', 'Discount', 'Profit']


In [28]:
# ---------- Parse date column ----------
if DATE_COL and DATE_COL in df.columns:
    df[DATE_COL] = pd.to_datetime(df[DATE_COL], errors="coerce")
    print(f"Parsed '{DATE_COL}'. NaT count:", int(df[DATE_COL].isna().sum()))
else:
    DATE_COL = None

Parsed 'Order Date'. NaT count: 0


In [29]:
# ---------- Validate region column ----------
if REGION_COL and REGION_COL not in df.columns:
    print(f"Warning: REGION_COL '{REGION_COL}' not found in uploaded CSV. Proceeding without region-based splitting.")
    REGION_COL = None

In [30]:
# ---------- Detect numeric columns to scale ----------
# Exclude obvious ID/date/region fields
exclude_candidates = {"row id", "row_id", "order id", "order_id", "orderid"}
exclude_cols = []
for c in df.columns:
    if c.lower() in exclude_candidates:
        exclude_cols.append(c)
if DATE_COL:
    exclude_cols.append(DATE_COL)
if REGION_COL:
    exclude_cols.append(REGION_COL)

numeric_cols = [c for c in df.columns if df[c].dtype.kind in "fi" and c not in exclude_cols]
print("Numeric columns detected for scaling:", numeric_cols)
if not numeric_cols:
    raise SystemExit("No numeric columns found to scale. Adjust DATE_COL/REGION_COL or check CSV.")

Numeric columns detected for scaling: ['Postal Code', 'Sales', 'Quantity', 'Discount', 'Profit']


In [31]:
# ---------- Choose scaler ----------
if SCALER_CHOICE == "standard":
    scaler = StandardScaler()
elif SCALER_CHOICE == "minmax":
    scaler = MinMaxScaler()
elif SCALER_CHOICE == "robust":
    scaler = RobustScaler()
else:
    raise ValueError("SCALER_CHOICE must be 'standard', 'minmax', or 'robust'.")

print("Using scaler:", SCALER_CHOICE)

Using scaler: standard


In [32]:
# ---------- Time-safe splitting function ----------
def time_train_test_split(df_in, date_col=None, region_col=None, test_size=0.2):
    if region_col and region_col in df_in.columns:
        train_idx, test_idx = [], []
        for region, group in df_in.groupby(region_col):
            if date_col and date_col in group.columns:
                group_sorted = group.sort_values(date_col)
            else:
                group_sorted = group  # assume already ordered
            n = len(group_sorted)
            k = max(1, int(n * (1 - test_size)))
            train_idx += group_sorted.index[:k].tolist()
            test_idx += group_sorted.index[k:].tolist()
        return sorted(train_idx), sorted(test_idx)
    elif date_col and date_col in df_in.columns:
        df_sorted = df_in.sort_values(date_col)
        n = len(df_sorted)
        k = max(1, int(n * (1 - test_size)))
        return df_sorted.index[:k].tolist(), df_sorted.index[k:].tolist()
    else:
        # fallback random split (not time-safe)
        return train_test_split(df_in.index, test_size=test_size, random_state=42)

In [33]:
# ---------- Fit scaler & transform ----------
df_scaled = df.copy()

if TIME_SAFE:
    train_idx, test_idx = time_train_test_split(df, date_col=DATE_COL, region_col=REGION_COL, test_size=TEST_SIZE)
    print("Time-safe split sizes -> train:", len(train_idx), " test:", len(test_idx))
    # Fit scaler on TRAIN numeric values only
    scaler.fit(df.loc[train_idx, numeric_cols].astype(float).values)
    # Transform both sets
    df_scaled.loc[train_idx, numeric_cols] = scaler.transform(df.loc[train_idx, numeric_cols].astype(float).values)
    df_scaled.loc[test_idx, numeric_cols]  = scaler.transform(df.loc[test_idx, numeric_cols].astype(float).values)
else:
    # Fit on full dataset (not recommended for forecasting/time-series)
    scaler.fit(df[numeric_cols].astype(float).values)
    df_scaled[numeric_cols] = scaler.transform(df[numeric_cols].astype(float).values)

Time-safe split sizes -> train: 1694  test: 427


/tmp/ipython-input-2222932258.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[-0.41531959 -0.41531959 -0.69812926 ...  0.45667431 -0.70219646
  1.14347119]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_scaled.loc[train_idx, numeric_cols] = scaler.transform(df.loc[train_idx, numeric_cols].astype(float).values)
/tmp/ipython-input-2222932258.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[-0.81880195 -0.37064347  0.5256735  ... -1.26696043 -0.37064347
 -0.81880195]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_scaled.loc[train_idx, numeric_cols] = scaler.transform(df.loc[train_idx, numeric_cols].astype(float).values)


In [34]:
# ---------- Save outputs ----------
scaled_csv_path = os.path.join(OUTPUT_DIR, "stores_sales_scaled.csv")
scaler_path = os.path.join(OUTPUT_DIR, f"scaler_{SCALER_CHOICE}_{'timesafe' if TIME_SAFE else 'full'}.joblib")

df_scaled.to_csv(scaled_csv_path, index=False)
joblib.dump(scaler, scaler_path)

print("\nSaved scaled CSV to:", scaled_csv_path)
print("Saved fitted scaler to:", scaler_path)


Saved scaled CSV to: /content/scaling_output/stores_sales_scaled.csv
Saved fitted scaler to: /content/scaling_output/scaler_standard_timesafe.joblib


In [35]:
# ---------- Show scaled output in this notebook (terminal-like) ----------
print("\n--- Preview: first 10 rows (scaled numeric columns) ---")
display(df_scaled[numeric_cols].head(10))


--- Preview: first 10 rows (scaled numeric columns) ---


,Postal Code,Sales,Quantity,Discount,Profit
0,-0.415320,-0.180284,-0.818802,-0.957226,0.233143
1,-0.415320,0.752037,-0.370643,-0.957226,1.544680
2,-0.698129,1.199644,0.525673,1.519754,-2.903771
3,1.062903,-0.603020,1.421990,-0.957226,0.028337
4,1.062903,2.684688,2.318307,0.143654,0.553487
5,-1.138100,-0.558362,-0.818802,0.694094,-0.083787
6,0.877396,1.372334,-0.370643,-0.957226,1.697360
7,-1.138100,5.416792,1.421990,1.794974,-12.367571
8,-1.138100,-0.453564,-0.370643,0.143654,0.038344
9,0.598685,-0.321209,0.525673,2.345414,-1.168514


In [36]:
# ---------- Robust inverse-transform & display (handles single-col and multi-col) ----------
row_idx = df_scaled.index[0]

In [37]:
# Ensure we get a 2D numpy array for single-row selection
scaled_row = df_scaled.loc[row_idx, numeric_cols].to_numpy(dtype=float)
scaled_row = np.atleast_2d(scaled_row)         # ensures shape is (1, n) even if n==1

In [38]:
# Inverse transform safely
original_est = scaler.inverse_transform(scaled_row)
original_est = np.atleast_2d(np.asarray(original_est))

In [39]:
# Convert to lists for friendly printing (works for n=1 as well)
scaled_list = np.round(scaled_row.flatten(), 6).tolist()
original_est_list = np.round(original_est.flatten(), 6).tolist()
original_source_list = np.round(df.loc[row_idx, numeric_cols].astype(float).to_numpy().flatten(), 6).tolist()

In [40]:
print("\nExample inverse-transform for row index", row_idx)
print("Scaled (vector):", scaled_list)
print("Recovered approx original (inverse_transform):", original_est_list)
print("Original values in uploaded CSV (for comparison):", original_source_list)


Example inverse-transform for row index 0
Scaled (vector): [-0.41532, -0.180284, -0.818802, -0.957226, 0.233143]
Recovered approx original (inverse_transform): [42420.0, 261.96, 2.0, 0.0, 41.9136]
Original values in uploaded CSV (for comparison): [42420.0, 261.96, 2.0, 0.0, 41.9136]


--- Notes ---
1. TIME_SAFE = True  (fit scaler on training subset only to avoid leakage).
2. Scaler used: standard
3. Scaled CSV and scaler saved under: /content/scaling_output
4. To reuse scaler later (example):
   from joblib import load; s = load('/content/scaling_output/scaler_standard_timesafe.joblib'); Xs = s.transform(X_new[numeric_cols])

In [41]:
print("\n--- Full scaled dataset ---")
display(df_scaled)


--- Full scaled dataset ---


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,-0.415320,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,-0.180284,-0.818802,-0.957226,0.233143
1,2,CA-2016-152156,2016-11-08,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,-0.415320,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",0.752037,-0.370643,-0.957226,1.544680
2,4,US-2015-108966,2015-10-11,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,-0.698129,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,1.199644,0.525673,1.519754,-2.903771
3,6,CA-2014-115812,2014-06-09,6/14/2014,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,1.062903,West,FUR-FU-10001487,Furniture,Furnishings,Eldon Expressions Wood and Plastic Desk Access...,-0.603020,1.421990,-0.957226,0.028337
4,11,CA-2014-115812,2014-06-09,6/14/2014,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,1.062903,West,FUR-TA-10001539,Furniture,Tables,Chromcraft Rectangular Conference Tables,2.684688,2.318307,0.143654,0.553487
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2116,9963,CA-2015-168088,2015-03-19,3/22/2015,First Class,CM-12655,Corinna Mitchell,Home Office,United States,Houston,...,0.659568,Central,FUR-BO-10004218,Furniture,Bookcases,Bush Heritage Pine Collection 5-Shelf Bookcase...,0.060752,0.077515,0.804182,-0.575799
2117,9965,CA-2016-146374,2016-12-05,12/10/2016,Second Class,HE-14800,Harold Engle,Corporate,United States,Newark,...,-1.120372,East,FUR-FU-10002671,Furniture,Furnishings,Electrix 20W Halogen Replacement Bulb for Zoom...,-0.673363,-1.266960,-0.957226,-0.028780
2118,9981,US-2015-151435,2015-09-06,9/9/2015,Second Class,SW-20455,Shaun Weien,Consumer,United States,Lafayette,...,0.456674,South,FUR-TA-10001039,Furniture,Tables,KI Adjustable-Height Table,-0.529383,-1.266960,-0.957226,0.088761
2119,9990,CA-2014-110422,2014-01-21,1/23/2014,Second Class,TB-21400,Tom Boeckenhauer,Consumer,United States,Miami,...,-0.702196,South,FUR-FU-10001889,Furniture,Furnishings,Ultra Door Pull Handle,-0.649860,-0.370643,0.143654,-0.045974
